In [1]:
import pandas as pd
import boto3
import json

from multiprocessing import Pool

In [52]:
missing = pd.read_csv('data/sinergise_only.txt')

In [54]:
import requests
import boto3

bucket = 'sentinel-s2-l2a'
client = boto3.client('s3')

def get_errors(tileinfo, l2):
    errors = []
    if not l2.get('tileDataGeometry'):
        errors.append("No tileDataGeometry field")
    if l2['cloudyPixelPercentage'] == 0:
        try:
            url_l1 = "https://roda.sentinel-hub.com/sentinel-s2-l1c" + tileinfo.split(bucket)[1]
            l1 = requests.get(url_l1).json()
            if l1['cloudyPixelPercentage'] == 0:
                errors.append("Cloud value 0.0 in L1 and L2")
        except Exception as e:
            errors.append(f"L1 metadata {url_l1} not found.")
    return errors


def check_tile(tileinfo):
    url_l2 = "https://roda.sentinel-hub.com/sentinel-s2-l2a" + tileinfo.split(bucket)[1]
    errors = []
    try:
        l2 = requests.get(url_l2).json()
    except Exception as e:
        errors.append(f"L2 metadata {url_l2} not found.")
    if not errors:
        errors = get_errors(tileinfo, l2)

    return [tileinfo, errors]


p = Pool(100)
result = p.map(check_tile, missing['sinergise'])

In [56]:
output = {}
unresolved = []
for item in result:
    if item[1]:
        output[item[0]] = item[1]
    else:
        unresolved.append(item[0])
print(len(result))
print(len(output))
# with open('data/sinergise-qa-output.json', 'w') as outfile:
#     json.dump(output, outfile, indent=2)
# with open('data/sinergise-qa-todo.json', 'w') as outfile:
#     json.dump(unresolved, outfile, indent=2)